In [ ]:
# Copyright 2023 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Vertex AI模型花园- Mistral和Mixtral 8x7B模型

<table align="left">
  <td>
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/community/model_garden/model_garden_pytorch_mistral.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> 在Colab中运行
    </a>
  </td>
  <td>
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/community/model_garden/model_garden_pytorch_mistral.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo"> 查看在GitHub上
    </a>
  </td>
  <td>
    <a href="https://console.cloud.google.com/vertex-ai/notebooks/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/vertex-ai-samples/main/notebooks/community/model_garden/model_garden_pytorch_mistral.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo"> 在Vertex AI Workbench中打开
    </a>（推荐使用Python-3 CPU笔记本）
  </td>
</table>

## 概述

本笔记本演示了在Vertex AI中部署预构建的[Mistral](https://mistral.ai/)和Mixtral 8x7B模型。

### 目标

- 使用[vLLM](https://github.com/vllm-project/vllm)容器部署预构建的[Mistral模型](https://huggingface.co/mistralai)
    - [mistralai/Mistral-7B-v0.1](https://huggingface.co/mistralai/Mistral-7B-v0.1)：具有70亿参数的预训练生成文本模型
    - [mistralai/Mistral-7B-Instruct-v0.1](https://huggingface.co/mistralai/Mistral-7B-Instruct-v0.1)：Mistral-7B-v0.1生成文本模型的指导微调版本
    - [mistralai/Mistral-7B-Instruct-v0.2](https://huggingface.co/mistralai/Mistral-7B-Instruct-v0.2)：支持32k上下文长度的Mistral-7B-Instruct-v0.1改进的指导微调版本
- 使用[vLLM](https://github.com/vllm-project/vllm)容器部署预构建的[Mixtral 8x7B模型](https://huggingface.co/mistralai)
    - [mistralai/Mixtral-8x7B-v0.1](https://huggingface.co/mistralai/Mixtral-8x7B-v0.1)：具有8个分支的预训练专家混合（MoE）模型
    - [mistralai/Mixtral-8x7B-Instruct-v0.1](https://huggingface.co/mistralai/Mixtral-8x7B-Instruct-v0.1)：具有8个分支的Mixture of Experts（MoE）模型的指导微调版本

### 成本

本教程使用Google Cloud的计费组件：

* Vertex AI
* 云存储

了解[Vertex AI定价](https://cloud.google.com/vertex-ai/pricing)、[云存储定价](https://cloud.google.com/storage/pricing)、[云NL API定价](https://cloud.google.com/natural-language/pricing)以及使用[定价计算器](https://cloud.google.com/products/calculator/)根据您的预期使用量生成成本估算。

在你开始之前

### 仅限 Colab
对于 Colab 运行以下命令，如果您正在使用 Workbench 则跳过此部分。

In [ ]:
import sys

if "google.colab" in sys.modules:
    ! pip3 install --upgrade google-cloud-aiplatform
    from google.colab import auth as google_auth

    google_auth.authenticate_user()
    # Install gdown for downloading example training images.
    ! pip3 install gdown

    # Restart the notebook kernel after installs.
    import IPython

    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

### 安装依赖项

In [ ]:
! pip3 install transformers==4.36.0
! pip3 install accelerate==0.23.0

### 设置谷歌云项目

1. [选择或创建一个谷歌云项目](https://console.cloud.google.com/cloud-resource-manager)。当您第一次创建帐户时，您将获得300美元的免费信用用于计算/存储成本。

2. [确保您的项目启用了计费](https://cloud.google.com/billing/docs/how-to/modify-project)。

3. [启用Vertex AI API，Compute Engine API和Cloud Natural Language API](https://console.cloud.google.com/flows/enableapi?apiid=aiplatform.googleapis.com,compute_component,language.googleapis.com)。

4. [创建一个Cloud Storage存储桶](https://cloud.google.com/storage/docs/creating-buckets) 用于存储实验输出。

5. [创建一个服务账号](https://cloud.google.com/iam/docs/service-accounts-create#iam-service-accounts-create-console) 并分配 `Vertex AI User` 和 `Storage Object Admin` 角色，用于部署微调模型到Vertex AI终端。

定义环境变量

为实验环境设置以下变量。指定的云存储桶（`BUCKET_URI`）应位于指定的区域（`REGION`）。请注意，多区域存储桶（例如“us”）不被认为与由多区域范围（例如“us-central1”）覆盖的单个区域匹配。

In [ ]:
# Cloud project id.
PROJECT_ID = ""  # @param {type:"string"}

# The region you want to launch jobs in.
# Select region based on the accelerators and regions supported by Vertex AI Prediction
# https://cloud.google.com/vertex-ai/docs/predictions/configure-compute.
REGION = ""  # @param {type:"string"}

# The Cloud Storage bucket for storing experiments output.
# Start with gs:// prefix, e.g. gs://foo_bucket.
BUCKET_URI = "gs://"  # @param {type:"string"}

! gcloud config set project $PROJECT_ID

import os

STAGING_BUCKET = os.path.join(BUCKET_URI, "temporal")

# The service account looks like:
# '@.iam.gserviceaccount.com'
# Please go to https://cloud.google.com/iam/docs/service-accounts-create#iam-service-accounts-create-console
# and create service account with `Vertex AI User` and `Storage Object Admin` roles.
# The service account for deploying fine tuned model.
SERVICE_ACCOUNT = ""  # @param {type:"string"}# HuggingFace access token.
# Create a token at https://huggingface.co/settings/tokens.
# Accept the model terms of service for the model you wish to use.
HF_TOKEN = ""  # @param {type:"string"}

初始化 Vertex AI API

In [ ]:
from google.cloud import aiplatform

aiplatform.init(project=PROJECT_ID, location=REGION, staging_bucket=STAGING_BUCKET)

### 定义常数

In [ ]:
# The pre-built serving docker images with vLLM
VLLM_DOCKER_URI = "us-docker.pkg.dev/vertex-ai/vertex-vision-model-garden-dockers/pytorch-vllm-serve:20240313_0916_RC00"

### 定义常见函数

In [ ]:
import os
from datetime import datetime
from typing import Tuple

from google.cloud import aiplatform


def get_job_name_with_datetime(prefix: str) -> str:
    """Gets the job name with date time when triggering training or deployment
    jobs in Vertex AI.
    """
    return prefix + datetime.now().strftime("_%Y%m%d_%H%M%S")


def deploy_model_vllm(
    model_name: str,
    model_id: str,
    service_account: str,
    machine_type: str = "g2-standard-8",
    accelerator_type: str = "NVIDIA_L4",
    accelerator_count: int = 1,
    max_model_len: int = 4096,
    gpu_memory_utilization: float = 0.9,
    use_openai_server: bool = False,
    use_chat_completions_if_openai_server: bool = False,
    huggingface_token: str = "",
) -> Tuple[aiplatform.Model, aiplatform.Endpoint]:
    """Deploys Mistral models with vLLM on Vertex AI.

    Args:
        model_name: Display name of the model.
        model_id: Model ID or path to model weights.
        service_account: Service account for model uploading and deployment.
        machine_type: Deployment machine type.
        accelerator_type: Deployment accelerator type.
        accelerator_count: Number of accelerators to use.
        max_model_len: Maximum model length.
        gpu_memory_utilization: Fraction of GPU memory to be used for the model
            executor.
        use_openai_server: Whether to use the OpenAI-format vLLM model server.
        use_chat_completions_if_openai_server: If the OpenAI model server is
            used, whether to use the chat completion API as opposed to the text
            completion API. The vLLM text completion API mimics the OpenAI text
            completion API:
            https://platform.openai.com/docs/api-reference/completions/create.
            It has two required parameters: the model ID to direct requests to
            and the prompt. The response includes a "choices" field that
            contains the generated text and a "usage" field that contains token
            counts. The vLLM chat completion API mimics the OpenAI chat
            completion API:
            https://platform.openai.com/docs/api-reference/chat/create. It has
            two required parameters: the model ID to direct requests to and
            "messages" which is a sequence of system/user/assistant/tool
            messages that can represent a multi-turn chat conversation. The
            response includes a "choices" field that contains the generated
            message from a role and a "usage" field that contains token counts.
        huggingface_token: Huggingface token for accessing the model.

    Returns:
        Model instance and endpoint instance.
    """
    endpoint = aiplatform.Endpoint.create(display_name=f"{model_name}-endpoint")

    dtype = "bfloat16"
    if accelerator_type in ["NVIDIA_TESLA_T4", "NVIDIA_TESLA_V100"]:
        dtype = "float16"

    vllm_args = [
        "--host=0.0.0.0",
        "--port=7080",
        f"--model={model_id}",
        f"--tensor-parallel-size={accelerator_count}",
        "--swap-space=16",
        f"--dtype={dtype}",
        f"--gpu-memory-utilization={gpu_memory_utilization}",
        f"--max-model-len={max_model_len}",
        "--disable-log-stats",
    ]
    serving_env = {
        "MODEL_ID": model_id,
        "DEPLOY_SOURCE": "notebook",
        "HF_TOKEN": huggingface_token,
    }
    if use_openai_server:
        if use_chat_completions_if_openai_server:
            serving_container_predict_route = "/v1/chat/completions"
        else:
            serving_container_predict_route = "/v1/completions"
    else:
        serving_container_predict_route = "/generate"
    model = aiplatform.Model.upload(
        display_name=model_name,
        serving_container_image_uri=VLLM_DOCKER_URI,
        serving_container_command=[
            "python",
            "-m",
            (
                "vllm.entrypoints.api_server"
                if not use_openai_server
                else "vllm.entrypoints.openai.api_server"
            ),
        ],
        serving_container_args=vllm_args,
        serving_container_ports=[7080],
        serving_container_predict_route=serving_container_predict_route,
        serving_container_health_route="/health",
        serving_container_environment_variables=serving_env,
    )

    model.deploy(
        endpoint=endpoint,
        machine_type=machine_type,
        accelerator_type=accelerator_type,
        accelerator_count=accelerator_count,
        deploy_request_timeout=1800,
        service_account=service_account,
    )
    return model, endpoint

使用预构建的Mistral和Mixtral模型在本地进行推理

您至少需要24GB的内存才能运行Mistral-7B的推理。您可以在本地运行，或者在Vertex AI预测端点上使用以下任何规格：
- g2-standard-8带有1个L4 GPU
- n1-standard-16带有2个V100 GPU
- n1-standard-16带有2个T4 GPU
- a2-highgpu-1g带有1个A100 GPU

您至少需要96GB的内存才能运行Mixtral 8x7B的推理。您可以在本地运行，或者在Vertex AI预测端点上使用以下任何规格：
- g2-standard-96带有8个L4 GPU
- n1-standard-32带有8个V100 GPU
- n1-standard-32带有8个T4 GPU
- a2-highgpu-4g带有4个A100 GPU

In [ ]:
%%time
import torch
import transformers
from transformers import AutoModelForCausalLM, AutoTokenizer

device = "cuda"  # the device to load the model onto
model_name = "mistralai/Mistral-7B-v0.1"  # @param ["mistralai/Mistral-7B-v0.1", "mistralai/Mistral-7B-Instruct-v0.1", "mistralai/Mistral-7B-Instruct-v0.2", "mistralai/Mixtral-8x7B-v0.1", "mistralai/Mixtral-8x7B-Instruct-v0.1"]
model = AutoModelForCausalLM.from_pretrained(
    model_name, device_map="auto", return_dict=True, torch_dtype=torch.float16
)
tokenizer = AutoTokenizer.from_pretrained(model_name)

pipeline = transformers.pipeline("text-generation", model=model, tokenizer=tokenizer)

prompt = "My favourite condiment is"

sequences = pipeline(
    prompt,
    max_length=200,
    do_sample=True,
    top_k=10,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id,
)

for seq in sequences:
    print(f"Result: {seq['generated_text']}")

## 使用vLLM部署预构建的Mistral模型

本部分在Vertex端点上部署了预构建的Mistral模型，使用[vLLM](https://github.com/vllm-project/vllm)。模型部署步骤将需要大约15分钟完成。

vLLM是一个高度优化的LLM服务框架，可以显著提高服务吞吐量。您拥有的QPS越高，使用vLLM获得的好处就越多。

设置预先构建的模型ID。

In [ ]:
prebuilt_model_id = "mistralai/Mistral-7B-v0.1"  # @param ["mistralai/Mistral-7B-v0.1", "mistralai/Mistral-7B-Instruct-v0.1", "mistralai/Mistral-7B-Instruct-v0.2"]

In [ ]:
# Find Vertex AI prediction supported accelerators and regions in
# https://cloud.google.com/vertex-ai/docs/predictions/configure-compute.

# Sets 1 L4 to deploy Mistral 7B.
machine_type = "g2-standard-8"
accelerator_type = "NVIDIA_L4"
accelerator_count = 1

# Sets 2 V100s to deploy Mistral 7B.
# machine_type = "n1-standard-16"
# accelerator_type = "NVIDIA_TESLA_V100"
# accelerator_count = 2

# Sets 2 T4s to deploy Mistral 7B.
# machine_type = "n1-standard-16"
# accelerator_type = "NVIDIA_TESLA_T4"
# accelerator_count = 2

# Sets 1 A100 (40G) to deploy Mistral 7B.
# machine_type = "a2-highgpu-1g"
# accelerator_type = "NVIDIA_TESLA_A100"
# accelerator_count = 1

# Larger setting of `max-model-len` can lead to higher requirements on
# `gpu-memory-utilization` and GPU configuration. Larger setting of
# `gpu-memory-utilization` increases the risk of running out of GPU memory with
# long prompts.
max_model_len = 4096
gpu_memory_utilization = 0.9

model, endpoint = deploy_model_vllm(
    model_name=get_job_name_with_datetime(prefix="mistral-serve-vllm"),
    model_id=prebuilt_model_id,
    service_account=SERVICE_ACCOUNT,
    machine_type=machine_type,
    accelerator_type=accelerator_type,
    accelerator_count=accelerator_count,
    max_model_len=max_model_len,
    gpu_memory_utilization=gpu_memory_utilization,
    use_openai_server=False,
    use_chat_completions_if_openai_server=False,
    huggingface_token=HF_TOKEN,
)

请注意：如果在向端点发送请求时出现`ServiceUnavailable: 503 502:Bad Gateway`错误，那么模型服务器很可能仍在初始化中。请稍后重试。

请注意：如果在部署过程中收到`InternalServerError: 500 System error`，很可能操作失败是因为资源不可用。请重试或使用不同的加速器类型。

一旦部署成功，您可以使用文本提示向端点发送请求。

### 运行示例提示

In [ ]:
# Loads an existing endpoint instance using the endpoint name:
# - Using `endpoint_name = endpoint.name` allows us to get the endpoint name of
#   the endpoint `endpoint` created in the cell above.
# - Alternatively, you can set `endpoint_name = "1234567890123456789"` to load
#   an existing endpoint with the ID 1234567890123456789.
# You may uncomment the code below to load an existing endpoint.

# endpoint_name = endpoint_without_peft.name
# # endpoint_name = ""  # @param {type:"string"}
# aip_endpoint_name = (
#     f"projects/{PROJECT_ID}/locations/{REGION}/endpoints/{endpoint_name}"
# )
# endpoint = aiplatform.Endpoint(aip_endpoint_name)

instances = [
    {
        "prompt": "My favourite condiment is",
        "n": 1,
        "max_tokens": 200,
        "temperature": 1.0,
        "top_p": 1.0,
        "top_k": 10,
    },
]
response = endpoint.predict(instances=instances)

for prediction in response.predictions:
    print(prediction)

# Reference the following code for using the OpenAI vLLM server.
# import json
# response = endpoint.raw_predict(
#     body=json.dumps({
#         "model": prebuilt_model_id,
#         "prompt": "My favourite condiment is",
#         "n": 1,
#         "max_tokens": 200,
#         "temperature": 1.0,
#         "top_p": 1.0,
#         "top_k": 10,
#     }),
#     headers={"Content-Type": "application/json"},
# )
# print(response.json())

## 使用vLLM部署预构建的Mixtral 8x7B模型

本部分将在Vertex端点上部署预构建的Mixtral 8x7B模型，使用[vLLM](https://github.com/vllm-project/vllm)。模型部署步骤将需要大约40分钟完成。

vLLM是一个高度优化的LLM服务框架，可以显著提高服务吞吐量。您拥有的QPS越高，使用vLLM带来的好处就越多。

设置预建模型ID。

In [ ]:
prebuilt_model_id = "mistralai/Mixtral-8x7B-v0.1"  # @param ["mistralai/Mixtral-8x7B-v0.1", "mistralai/Mixtral-8x7B-Instruct-v0.1"]

In [ ]:
# Find Vertex AI prediction supported accelerators and regions in
# https://cloud.google.com/vertex-ai/docs/predictions/configure-compute.

# Sets 8 L4s to deploy Mixtral 8x7B.
machine_type = "g2-standard-96"
accelerator_type = "NVIDIA_L4"
accelerator_count = 8

# Sets 4 A100s (40G) to deploy Mixtral 8x7B.
# machine_type = "a2-highgpu-4g"
# accelerator_type = "NVIDIA_TESLA_A100"
# accelerator_count = 4

# Larger setting of `max-model-len` can lead to higher requirements on
# `gpu-memory-utilization` and GPU configuration. Larger setting of
# `gpu-memory-utilization` increases the risk of running out of GPU memory with
# long prompts.
max_model_len = 4096
gpu_memory_utilization = 0.85

model, endpoint = deploy_model_vllm(
    model_name=get_job_name_with_datetime(prefix="mixtral-serve-vllm"),
    model_id=prebuilt_model_id,
    service_account=SERVICE_ACCOUNT,
    machine_type=machine_type,
    accelerator_type=accelerator_type,
    accelerator_count=accelerator_count,
    max_model_len=max_model_len,
    gpu_memory_utilization=gpu_memory_utilization,
    use_openai_server=False,
    use_chat_completions_if_openai_server=False,
    huggingface_token=HF_TOKEN,
)

注意：如果在向端点发送请求时看到`ServiceUnavailable: 503 502:Bad Gateway`错误，那么模型服务器可能仍在初始化中。请稍后重试。

注意：如果在部署过程中收到`InternalServerError: 500 System error`，很可能是由于资源不足导致操作失败。请重试或使用不同的加速器类型。

一旦部署成功，您可以通过文本提示向端点发送请求。

### 运行示例提示

In [ ]:
# Loads an existing endpoint instance using the endpoint name:
# - Using `endpoint_name = endpoint.name` allows us to get the endpoint name of
#   the endpoint `endpoint` created in the cell above.
# - Alternatively, you can set `endpoint_name = "1234567890123456789"` to load
#   an existing endpoint with the ID 1234567890123456789.
# You may uncomment the code below to load an existing endpoint.

# endpoint_name = endpoint_without_peft.name
# # endpoint_name = ""  # @param {type:"string"}
# aip_endpoint_name = (
#     f"projects/{PROJECT_ID}/locations/{REGION}/endpoints/{endpoint_name}"
# )
# endpoint = aiplatform.Endpoint(aip_endpoint_name)

instances = [
    {
        "prompt": "What is a car?",
        "max_tokens": 50,
        "temperature": 1.0,
        "top_p": 1.0,
        "top_k": 10,
    },
]
response = endpoint.predict(instances=instances)

for prediction in response.predictions:
    print(prediction)

# Reference the following code for using the OpenAI vLLM server.
# import json
# response = endpoint.raw_predict(
#     body=json.dumps({
#         "model": prebuilt_model_id,
#         "prompt": "My favourite condiment is",
#         "n": 1,
#         "max_tokens": 200,
#         "temperature": 1.0,
#         "top_p": 1.0,
#         "top_k": 10,
#     }),
#     headers={"Content-Type": "application/json"},
# )
# print(response.json())

清理

要清理此项目中使用的所有Google Cloud资源，您可以[删除用于本教程的Google Cloud项目](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects)。

否则，您可以删除本教程中创建的各个资源。

### 下线模型并删除端点

In [ ]:
# Set this flag to delete endpoint including undeploying models
delete_endpoint = False

In [ ]:
def list_endpoints():
    return [
        (r.name, r.display_name)
        for r in aiplatform.Endpoint.list()
        if r.display_name.startswith("mistral-serve-vllm")
        or r.display_name.startswith("mixtral-serve-vllm")
    ]

In [ ]:
# Delete the endpoint using the Vertex AI fully qualified identifier for the endpoint
try:
    if delete_endpoint:
        endpoints = list_endpoints()
        for endpoint_id, endpoint_name in endpoints:
            endpoint = aiplatform.Endpoint(endpoint_id)
            print(
                f"Undeploying all deployed models and deleting endpoint {endpoint_id} [{endpoint_name}]"
            )
            endpoint.delete(force=True)
except Exception as e:
    print(e)

### 删除云存储存储桶

In [ ]:
import os

delete_bucket = False

job.delete()

if delete_bucket or os.getenv("ID_TESTING"):
    ! gsutil rm -rf {BUCKET_URI}